In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# the number of segments; when we deploy repeater
N = np.array([1,2,4,8,16,32,64,128])

# metal parameter
metal_r = 120 / 1e-6
metal_c = (0.2) * 1e-15 / 1e-6
WL_length = 256 * 1e-6

# cell parameter
cell_num = 256
cell_c = 1e-15
cell_C = cell_c * cell_num

###################################################################
### TR/Logic Gates parameters: based on P/N ratio 7:3 (2.57:1.1)###
###################################################################

# TR parameter
# gate cap
tau = 127/30 * 1e-12 #measured FO1 delay : tau(1+pinv)
TR_C = (1.5/1.1) * 1e-15 # measured
#tau = 3 * TR_C * TR_R
TR_R = tau / (3*TR_C)




# Logic Gates
g_inv, p_inv = 1, 323/127

g_nand2, g_nand3, g_nand6 = 4/3 , 5/3, 8/3
p_nand2, p_nand3, p_nand6 = p_inv*2, p_inv*3, p_inv*6

g_nor2, g_nor3 = 5/3, 7/3
p_nor2, p_nor3 = p_inv*2, p_inv*3

# Decoder Logic
G_nand6_inv = g_nand6 * g_inv
P_nand6_inv = p_nand6 + p_inv
G_nand3_nor2 = g_nand3 * g_nor2
P_nand3_nor2 = p_nand3 + p_nor2
B = 32 # critical path: CLK signal

In [3]:
# delay of decoder
WL_R = metal_r * WL_length
WL_C = metal_c * WL_length

# NMOS width of repeater derived from RC delay of N segments
W = ( N*(TR_R*(metal_c + 1e-15))/(3*metal_r*TR_C) ) ** (1/2)

C_out = 3 * W * TR_C
C_in = 5 * 1e-15 * 32
H = C_out/C_in

G = np.array([G_nand6_inv, G_nand3_nor2])
F = np.outer(G,H)*B

F = pd.DataFrame(data = F, columns = [1,2,4,8,16,32,64,128], index=['NAND6-INV','NAND3-NOR2'])
stages = np.log(F)/np.log(4)
stages

,1,2,4,8,16,32,64,128
NAND6-INV,0.251246,0.501246,0.751246,1.001246,1.251246,1.501246,1.751246,2.001246
NAND3-NOR2,0.280693,0.530693,0.780693,1.030693,1.280693,1.530693,1.780693,2.030693


In [4]:
#with 2 stages
f_2 = F ** (1/2)
delay_decoder_stage2 = 2*f_2
delay_decoder_stage2.loc['NAND6-INV'] = delay_decoder_stage2.loc['NAND6-INV'] + P_nand6_inv
delay_decoder_stage2.loc['NAND3-NOR2'] = delay_decoder_stage2.loc['NAND3-NOR2'] + P_nand3_nor2
delay_decoder_stage2 = delay_decoder_stage2 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage2','NAND3-NOR2':'NAND3-NOR2 Stage2'}
delay_decoder_stage2.rename(index = new_row_names, inplace = True)
delay_decoder_stage2

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,8.544399e-11,8.735069e-11,8.961815e-11,9.231464e-11,9.552131e-11,9.933471e-11,1.038696e-10,1.092626e-10
NAND3-NOR2 Stage2,6.411846e-11,6.606448e-11,6.837870e-11,7.113078e-11,7.440358e-11,7.829562e-11,8.292406e-11,8.842823e-11


In [5]:
#with 4 stages
f_4 = F ** (1/4)
delay_decoder_stage4 = 4*f_4
delay_decoder_stage4.loc['NAND6-INV'] = delay_decoder_stage4.loc['NAND6-INV'] + P_nand6_inv + 2 * p_inv
delay_decoder_stage4.loc['NAND3-NOR2'] = delay_decoder_stage4.loc['NAND3-NOR2'] + P_nand3_nor2 + 2 * p_inv
delay_decoder_stage4 = delay_decoder_stage4 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage4','NAND3-NOR2':'NAND3-NOR2 Stage4'}
delay_decoder_stage4.rename(index = new_row_names, inplace = True)
delay_decoder_stage4

,1,2,4,8,16,32,64,128
NAND6-INV Stage4,1.153739e-10,1.170459e-10,1.188693e-10,1.208577e-10,1.230261e-10,1.253907e-10,1.279693e-10,1.307813e-10
NAND3-NOR2 Stage4,9.403008e-11,9.571926e-11,9.756133e-11,9.957012e-11,1.017607e-10,1.041496e-10,1.067547e-10,1.095955e-10


In [6]:
#with 6 stages
f_6 = F ** (1/6)
delay_decoder_stage6 = 6*f_6
delay_decoder_stage6.loc['NAND6-INV'] = delay_decoder_stage6.loc['NAND6-INV'] + P_nand6_inv + 4 * p_inv
delay_decoder_stage6.loc['NAND3-NOR2'] = delay_decoder_stage6.loc['NAND3-NOR2'] + P_nand3_nor2 + 4 * p_inv
delay_decoder_stage6 = delay_decoder_stage6 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage6','NAND3-NOR2':'NAND3-NOR2 Stage6'}
delay_decoder_stage6.rename(index = new_row_names, inplace = True)
delay_decoder_stage6

,1,2,4,8,16,32,64,128
NAND6-INV Stage6,1.453514e-10,1.469521e-10,1.486479e-10,1.504445e-10,1.523480e-10,1.543647e-10,1.565013e-10,1.587649e-10
NAND3-NOR2 Stage6,1.240019e-10,1.256134e-10,1.273208e-10,1.291297e-10,1.310462e-10,1.330767e-10,1.352278e-10,1.375069e-10


In [7]:
#with 8 stages
f_8 = F ** (1/8)
delay_decoder_stage8 = 8*f_8
delay_decoder_stage8.loc['NAND6-INV'] = delay_decoder_stage8.loc['NAND3-NOR2'] + P_nand6_inv + 6 * p_inv
delay_decoder_stage8.loc['NAND3-NOR2'] = delay_decoder_stage8.loc['NAND3-NOR2'] + P_nand3_nor2 + 6 * p_inv
delay_decoder_stage8 = delay_decoder_stage8 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage8','NAND3-NOR2':'NAND3-NOR2 Stage8'}
delay_decoder_stage8.rename(index = new_row_names, inplace = True)
delay_decoder_stage8

,1,2,4,8,16,32,64,128
NAND6-INV Stage8,1.755213e-10,1.770955e-10,1.787393e-10,1.804559e-10,1.822485e-10,1.841205e-10,1.860754e-10,1.881168e-10
NAND3-NOR2 Stage8,1.539880e-10,1.555621e-10,1.572060e-10,1.589226e-10,1.607152e-10,1.625872e-10,1.645420e-10,1.665835e-10


In [8]:
delay_decoder = pd.concat([delay_decoder_stage2, delay_decoder_stage4, delay_decoder_stage6, delay_decoder_stage8])
delay_decoder

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,8.544399e-11,8.735069e-11,8.961815e-11,9.231464e-11,9.552131e-11,9.933471e-11,1.038696e-10,1.092626e-10
NAND3-NOR2 Stage2,6.411846e-11,6.606448e-11,6.837870e-11,7.113078e-11,7.440358e-11,7.829562e-11,8.292406e-11,8.842823e-11
NAND6-INV Stage4,1.153739e-10,1.170459e-10,1.188693e-10,1.208577e-10,1.230261e-10,1.253907e-10,1.279693e-10,1.307813e-10
NAND3-NOR2 Stage4,9.403008e-11,9.571926e-11,9.756133e-11,9.957012e-11,1.017607e-10,1.041496e-10,1.067547e-10,1.095955e-10
NAND6-INV Stage6,1.453514e-10,1.469521e-10,1.486479e-10,1.504445e-10,1.523480e-10,1.543647e-10,1.565013e-10,1.587649e-10
NAND3-NOR2 Stage6,1.240019e-10,1.256134e-10,1.273208e-10,1.291297e-10,1.310462e-10,1.330767e-10,1.352278e-10,1.375069e-10
NAND6-INV Stage8,1.755213e-10,1.770955e-10,1.787393e-10,1.804559e-10,1.822485e-10,1.841205e-10,1.860754e-10,1.881168e-10
NAND3-NOR2 Stage8,1.539880e-10,1.555621e-10,1.572060e-10,1.589226e-10,1.607152e-10,1.625872e-10,1.645420e-10,1.665835e-10


In [9]:
segment_delay = (TR_R/W) * ( (3*TR_C*p_inv*W) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * ( (3*TR_C*W) + cell_C/(2*N) + WL_C/(2*N) )
last_segment_delay = (TR_R/W) * ( (3*TR_C*p_inv*W) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * (cell_C/(2*N) + WL_C/(2*N))
repeater_delay = 15 *1e-12 #측정
delay_WL = (N-1) * segment_delay + last_segment_delay   + N * repeater_delay
delay_WL

array([5.23395368e-09, 2.81895823e-09, 1.66261154e-09, 1.20063729e-09,
       1.19907583e-09, 1.63694366e-09, 2.69329071e-09, 4.83188653e-09])

In [10]:
delay_total = delay_decoder.copy()
delay_total = delay_total + delay_WL
delay_total

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,5.319398e-09,2.906309e-09,1.752230e-09,1.292952e-09,1.294597e-09,1.736278e-09,2.797160e-09,4.941149e-09
NAND3-NOR2 Stage2,5.298072e-09,2.885023e-09,1.730990e-09,1.271768e-09,1.273479e-09,1.715239e-09,2.776215e-09,4.920315e-09
NAND6-INV Stage4,5.349328e-09,2.936004e-09,1.781481e-09,1.321495e-09,1.322102e-09,1.762334e-09,2.821260e-09,4.962668e-09
NAND3-NOR2 Stage4,5.327984e-09,2.914677e-09,1.760173e-09,1.300207e-09,1.300837e-09,1.741093e-09,2.800045e-09,4.941482e-09
NAND6-INV Stage6,5.379305e-09,2.965910e-09,1.811259e-09,1.351082e-09,1.351424e-09,1.791308e-09,2.849792e-09,4.990651e-09
NAND3-NOR2 Stage6,5.357956e-09,2.944572e-09,1.789932e-09,1.329767e-09,1.330122e-09,1.770020e-09,2.828519e-09,4.969393e-09
NAND6-INV Stage8,5.409475e-09,2.996054e-09,1.841351e-09,1.381093e-09,1.381324e-09,1.821064e-09,2.879366e-09,5.020003e-09
NAND3-NOR2 Stage8,5.387942e-09,2.974520e-09,1.819818e-09,1.359560e-09,1.359791e-09,1.799531e-09,2.857833e-09,4.998470e-09


In [11]:
delay_total.describe()

,1,2,4,8,16,32,64,128
count,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00
mean,5.353682e-09,2.940384e-09,1.785904e-09,1.325991e-09,1.326710e-09,1.767109e-09,2.826274e-09,4.968017e-09
std,3.762337e-11,3.749196e-11,3.725346e-11,3.688524e-11,3.636109e-11,3.565116e-11,3.472231e-11,3.353948e-11
min,5.298072e-09,2.885023e-09,1.730990e-09,1.271768e-09,1.273479e-09,1.715239e-09,2.776215e-09,4.920315e-09
25%,5.325837e-09,2.912585e-09,1.758187e-09,1.298394e-09,1.299277e-09,1.739890e-09,2.799324e-09,4.941399e-09
50%,5.353642e-09,2.940288e-09,1.785707e-09,1.325631e-09,1.326112e-09,1.766177e-09,2.824889e-09,4.966031e-09
75%,5.381464e-09,2.968063e-09,1.813399e-09,1.353201e-09,1.353516e-09,1.793364e-09,2.851802e-09,4.992606e-09
max,5.409475e-09,2.996054e-09,1.841351e-09,1.381093e-09,1.381324e-09,1.821064e-09,2.879366e-09,5.020003e-09


In [12]:
W

array([0.64930219, 0.91825196, 1.29860437, 1.83650392, 2.59720874,
       3.67300783, 5.19441749, 7.34601566])

In [15]:
2*1.83650392

3.67300784

In [14]:
f_2

,1,2,4,8,16,32,64,128
NAND6-INV,1.190235,1.415436,1.683246,2.001729,2.380470,2.830872,3.366493,4.003457
NAND3-NOR2,1.214778,1.444623,1.717956,2.043006,2.429557,2.889246,3.435912,4.086012


In [16]:
C_out

array([2.65623622e-15, 3.75648528e-15, 5.31247243e-15, 7.51297056e-15,
       1.06249449e-14, 1.50259411e-14, 2.12498897e-14, 3.00518822e-14])